In [63]:
import consul

# print(help(consul.Consul))
c = consul.Consul(host='192.168.199.76',dc='uat')

print(help(c.catalog))

Help on Catalog in module consul.base object:

class Catalog(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self, agent)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  datacenters(self)
 |      Returns all the datacenters that are known by the Consul server.
 |  
 |  deregister(self, node, service_id=None, check_id=None, dc=None, token=None)
 |      A low level mechanism for directly removing entries in the catalog.
 |      It is usually recommended to use the agent APIs, as they are
 |      simpler and perform anti-entropy.
 |      
 |      *node* and *dc* specify which node on which datacenter to remove.
 |      If *service_id* and *check_id* are not provided, all associated
 |      services and checks are deleted. Otherwise only one of *service_id*
 |      and *check_id* should be provided and only that service or check
 |      will be removed.
 |      
 |      *token* is an optional `ACL token`_ to apply to this request.
 |      
 |   

In [33]:
d = c.catalog.datacenters()
print(d)

['uat']


In [34]:
c.catalog.nodes()

('107215',
 [{'ID': '7801f9e9-bcf2-1385-e590-0b330bf326e8',
   'Node': 'slq-als-1',
   'Address': '192.168.199.68',
   'Datacenter': 'uat',
   'TaggedAddresses': {'lan': '192.168.199.68', 'wan': '192.168.199.68'},
   'Meta': {'consul-network-segment': ''},
   'CreateIndex': 107212,
   'ModifyIndex': 107215}])

In [60]:
c.catalog.service('zuul')

('114393',
 [{'ID': '7801f9e9-bcf2-1385-e590-0b330bf326e8',
   'Node': 'slq-als-1',
   'Address': '192.168.199.68',
   'Datacenter': 'uat',
   'TaggedAddresses': {'lan': '192.168.199.68', 'wan': '192.168.199.68'},
   'NodeMeta': {'consul-network-segment': ''},
   'ServiceKind': '',
   'ServiceID': 'zuul-12081',
   'ServiceName': 'zuul',
   'ServiceTags': [],
   'ServiceAddress': '192.168.199.76',
   'ServiceMeta': {},
   'ServicePort': 12081,
   'ServiceEnableTagOverride': False,
   'ServiceProxyDestination': '',
   'ServiceConnect': {'Native': False, 'Proxy': None},
   'CreateIndex': 114392,
   'ModifyIndex': 114392}])

In [62]:
node = c.catalog.service('zuul')[1][0]['Node']
serverId = c.catalog.service('zuul')[1][0]['ServiceID']
dc = c.catalog.service('zuul')[1][0]['Datacenter']
print(node, serverId, dc)

slq-als-1 zuul-12081 uat


In [66]:
c.catalog.deregister('slq-als-1',u'zuul-12081')

True

## consul 集群状态健康

In [96]:
import consul

# print(help(consul.Consul))
c = consul.Consul(host='192.168.199.32',dc='uat')

class ConsulApi(object):
    def __init__(self):
    
    def cchecks(self):
        pass
    
    def cstatus(self):
        pass
        

#print(help(c.catalog))

c.status

# leader
c.status.leader()

# server
c.status.peers()

"""
{'service:Account:192.168.199.32:14994': {'Node': 'slq-uat-1',
  'CheckID': 'service:Account:192.168.199.32:14994',
  'Name': "Service 'Account' check",
  'Status': 'passing',
  'Notes': '',
  'Output': 'HTTP GET http://192.168.199.32:14994/api/Account/health/ping: 200 OK Output: "Account"',
  'ServiceID': 'Account:192.168.199.32:14994',
  'ServiceName': 'Account',
  'ServiceTags': [],
  'CreateIndex': 0,
  'ModifyIndex': 0},
"""
check_dict = c.agent.checks()

print(check_dict['service:Account:192.168.199.32:14994'])


#for n in check_dict:
#    print(check_dict[n]['ServiceName'])
#    print(check_dict[n]['Output'])
    

{'Node': 'slq-uat-1', 'CheckID': 'service:Account:192.168.199.32:14994', 'Name': "Service 'Account' check", 'Status': 'passing', 'Notes': '', 'Output': 'HTTP GET http://192.168.199.32:14994/api/Account/health/ping: 200 OK Output: "Account"', 'ServiceID': 'Account:192.168.199.32:14994', 'ServiceName': 'Account', 'ServiceTags': [], 'CreateIndex': 0, 'ModifyIndex': 0}


In [106]:

# ConnectionError

consul_members = c.agent.members()
for n in  consul_members:
    print(n['Name'])
    print(n['Status'])
    
print('__________')

slq-uat-1
1
slq-cache-1
1
slq-disque-1
1
slq-web-1
1
007-0000.local
4
ezbuy-zydeMacBook-Air.local
1
__________


In [108]:
c.health.node('slq-uat-1')

ReadTimeout: HTTPConnectionPool(host='192.168.199.32', port=8500): Read timed out. (read timeout=None)

In [ ]:
c.health.state('slq-uat-1')

## 服务监控

In [ ]:
import consul

# print(help(consul.Consul))
client = consul.Consul(host='192.168.199.76',dc='uat')


client.health.node(client.agent.members()[0]['Name'])